In [210]:
import autoreload
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [211]:
import requests
import base64
import datetime
from urllib.parse import urlencode
import pickle
import pandas as pd

# Authenticate & Create Relevant Functions for Spotify API

In [212]:
# Function to pull relevant song features
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
            
    def get_resource(self, lookup_id, resource_type, version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_audio_features(self, _id):
        return self.get_resource(_id, resource_type='audio-features')
    
    def get_audio_analysis(self, _id):
        return self.get_resource(_id, resource_type='audio-analysis')
    
    def search(self, query, search_type='artist' ): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": query, "type": search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()

# Pickle dump song dataset and save into dataframe

In [229]:
# Load Pickled Dataframe from Cleaning Notebook into a DataFrame
path = r"C:\Users\Andrew\Documents\Metis\TikTok_Song_Predictor\Pickle\Spotify_Pickle.pkl"

df = pickle.load(open(path,'rb'))

In [230]:
# Reset Index
df.reset_index(inplace=True)

# Extract Features

In [234]:
spotify = SpotifyAPI(client_id, client_secret)

In [238]:
# Function to extract song_id 

def extract_id(isrc):
    try: 
        song_id = spotify.search(f"isrc:{isrc}", search_type = "Track")['tracks']['items'][0]['id']
    except:
        song_id = {}
    return song_id

In [239]:
# Create an id column

df['spotify_uri'] = df['ISRC'].apply(lambda x : extract_id(x))

In [253]:
# Create audio analysis column
df['audio_analysis'] = df['spotify_uri'].apply(lambda x : spotify.get_audio_analysis(x))

In [254]:
# Create feature analysis column
df['feature_analysis'] = df['spotify_uri'].apply(lambda x : spotify.get_audio_features(x))

In [255]:
df.head()

,index,Playlist Add Date,Record Label,UPC,Album Name,ISRC,name,Peak Position,Peak Date,Rank,...,Artist(s),tiktok_track_id,Velocity,Historical Posts,TikTok Link,Release Date,Position Change,spotify_uri,audio_analysis,feature_analysis
0,0,2020-08-23T00:00:00.000Z,JVKE,193436230483,Upside Down,QM24S2008283,All TikTok Mashup (JVKE - Upside Down),1,2020-09-02,1,...,JVKE 🌩,All-TikTok-Mashup-JVKE-Upside-Down-68463501735...,,"100000, 300000",https://www.tiktok.com/music/All-TikTok-Mashup...,2020-08-17,23.0,5TpvLkESnw1g9wDz52efeO,"{'meta': {'analyzer_version': '4.0.0', 'platfo...","{'danceability': 0.88, 'energy': 0.501, 'key':..."
1,0,2020-11-03T00:00:00.000Z,ColliPark Music,859705372161,Alley: The Return of the Ying Yang Twins,USW9J1000104,Say I Yi Yi,1,2020-11-02,1,...,D-Roc,Say-I-Yi-Yi-6883981933215369989,,"105900, 148200, 177600, 229200, 200000, 200000",https://www.tiktok.com/music/Say-I-Yi-Yi-68839...,2002-03-25,2.0,46XvcuEZFFInDilbYUNHBn,"{'meta': {'analyzer_version': '4.0.0', 'platfo...","{'danceability': 0.782, 'energy': 0.627, 'key'..."
2,0,2020-11-01T00:00:00.000Z,MFLP,"829410801386, 888072351684, 600406849243, 8880...","Halloween Howls, Halloween Howls: Fun & Scary ...",USB440453203,"Spooky, Scary Skeletons",1,2020-10-31,1,...,Andrew Gold,Spooky-Scary-Skeletons-6602666381370460933,2.14286,"100000, 100000, 100000, 100000, 100000, 100000...",https://www.tiktok.com/music/Spooky-Scary-Skel...,2006-10-02,5.0,23V08GxMeaZNSf8Gy6KF6t,"{'meta': {'analyzer_version': '4.0.0', 'platfo...","{'danceability': 0.806, 'energy': 0.447, 'key'..."
3,0,2020-10-29T00:00:00.000Z,"Astroknot Sounds, Opposition",193436238021,Garage Room Freestyle,QM24S2009905,Garage Room Freestyle (Reel It In),1,2020-10-28,1,...,Number9ok,Garage-Room-Freestyle-Reel-It-In-6876123370077...,1.85714,"40000, 81500, 123900, 116700, 74700, 72700, 78...",https://www.tiktok.com/music/Garage-Room-Frees...,2020-10-13,14.0,0buOF1gvLBupoSZSaycYWD,"{'meta': {'analyzer_version': '4.0.0', 'platfo...","{'danceability': 0.849, 'energy': 0.195, 'key'..."
4,0,2020-10-26T00:00:00.000Z,Not specified,886448446026,"Ho, why is you here ?",USRC12002532,May I,1,2020-10-25,1,...,Flomilli,May-I-6852015601389438978,9.42857,"38200, 34100, 39700, 31200, 35900, 44400, 4080...",https://www.tiktok.com/music/May-I-68520156013...,2020-07-23,76.0,1GcXzijYzyfMq6wryZjYW5,"{'meta': {'analyzer_version': '4.0.0', 'platfo...","{'danceability': 0.896, 'energy': 0.78, 'key':..."


In [256]:
# Pickle the cleaned dataframe

# Designate path

path = r"C:\Users\Andrew\Documents\Metis\TikTok_Song_Predictor\Pickle\Feauture_Pickle.pkl"
with open(path, 'wb') as f:
    pickle.dump(df, f)